In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [36]:
default = pd.read_csv("C:\\Users\\hsbc_2019\\Desktop\\data analytics\\topic wise\\datasets\\bank\\bank.csv",delimiter=';')
default.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [37]:
default.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [38]:
default['y']=default['y'].replace(['no'],1)
default['y']=default['y'].replace(['yes'],0)

In [39]:
default = pd.get_dummies(default,columns=['job','marital','education','housing','loan','contact','month','poutcome'])


default.columns

Index(['age', 'default', 'balance', 'day', 'duration', 'campaign', 'pdays',
       'previous', 'y', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'housing_no', 'housing_yes', 'loan_no', 'loan_yes',
       'contact_cellular', 'contact_telephone', 'contact_unknown', 'month_apr',
       'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'poutcome_failure', 'poutcome_other', 'poutcome_success',
       'poutcome_unknown'],
      dtype='object')

In [40]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features_to_scale = ["age","y"]
scaled_values = scaler.fit_transform(default[features_to_scale])
default["norm_balance"] = scaled_values[:,0]
default["norm_income"] = scaled_values[:,1]

In [41]:
from sklearn.model_selection import train_test_split
default_train,default_test = train_test_split(default,test_size=0.2,random_state=100)

In [42]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=5,shuffle=False,random_state=100)
features = ["norm_balance","norm_income"]
target = ["default"]
knn_k_vals = [1,3,5,7,9,11]
avg_train_accuracy = []
avg_val_accuracy = []
X = default_train[features]
y = default_train[target]
#Taking each value of k from the list
for k in knn_k_vals: 
    # Iterating over each fold from kfold
    train_accuracy = []
    val_accuracy = []
    for i,(train,val) in enumerate(k_fold.split(X,y)): 
        #Train a model on the selected fold for a selected value of k
        model = KNeighborsClassifier(n_neighbors=k,metric="euclidean")
        model.fit(X.iloc[train],y.iloc[train])
        train_accuracy.append(model.score(X.iloc[train],y.iloc[train]))
        val_accuracy.append(model.score(X.iloc[val],y.iloc[val]))
    avg_train_accuracy.append(np.mean(train_accuracy))
    avg_val_accuracy.append(np.mean(val_accuracy))

In [43]:
performance_scores = pd.DataFrame(np.array([knn_k_vals,
                                            avg_train_accuracy,
                                            avg_val_accuracy]).T,
                     columns=["k","avg_train_accuracy","avg_val_accuracy"])
performance_scores

,k,avg_train_accuracy,avg_val_accuracy
0,1.0,0.977392,0.974003
1,3.0,0.982024,0.982025
2,5.0,0.982024,0.982025
3,7.0,0.982024,0.982025
4,9.0,0.982024,0.982025
5,11.0,0.982024,0.982025


In [44]:
best_k = performance_scores["k"][performance_scores["avg_val_accuracy"]==
                                 performance_scores["avg_val_accuracy"].max()]
best_k

1     3.0
2     5.0
3     7.0
4     9.0
5    11.0
Name: k, dtype: float64

#### Using the best k found to train a model

In [45]:
features = ["norm_balance","norm_income"]
target = ["default"]
model = KNeighborsClassifier(n_neighbors = 11, metric="euclidean")
model.fit(default_train[features],default_train[target])
train_accuracy = model.score(default_train[features],default_train[target])
test_accuracy = model.score(default_test[features],default_test[target])
print(train_accuracy,test_accuracy)

0.9820243362831859 0.9878453038674033
